# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary classifier.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title, and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use a random forest classifier, as well as another classifier of your choice; either logistic regression, SVM, or KNN. 

- **Question**: Why would we want this to be a classification problem?
- **Answer**: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Set up a request (using requests) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)
The URL here has many query parameters
- q for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- l for a location
- start for what result number to start on

In [1]:
URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=100"


In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import time
import numpy as np

In [3]:
html = requests.get(URL)
soup = BeautifulSoup(html.text)

/home/roland/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/roland/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Let's look at one result more closely. A single result looks like
```JSON
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&campaignid=serp-linkcompanyname&fromjk=2480d203f7e97210&jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a nobr element inside of a td element with class='snip.
- The title of a job is in a link with class set to jobtitle and a data-tn-element="jobTitle.
- The location is set in a span with class='location'.
- The company is set in a span with class='company'.

## Write 4 functions to extract these items (one function for each): location, company, job title, and salary.¶
Example
```python
def extract_location_from_result(result):
    return result.find ...
```

##### - Make sure these functions are robust and can handle cases where the data/field may not be available.
>- Remember to check if a field is empty or None for attempting to call methods on it
>- Remember to use try/except if you anticipate errors.

- **Test** the functions on the results above and simple examples

In [4]:
def extract_location_from_result(result):
    values = result.find_all(itemprop='addressLocality')
    if values != None:
        temp = list(map(lambda a: a.get_text(), values))
        if len(temp) == 1:
            return temp[0]
        else:
            return tuple(temp)
    else:
        return None


In [52]:

def extract_salary_from_result(result):
    values = result.findAll('span',class_='no-wrap')
    if values != None:
        # There are a number of salaries that are ranges, so I am spliting on white space and keeping
        # just the values that have a dollar sign.
        temp = list(map(lambda a: list(filter(lambda b: '$' in b, a.get_text().split(' '))),values))
        
        # Getting the first
        if len(temp) != 0:
            temp = temp[0]
            
        if len(temp) == 1:
            return clean_salary(temp[0])
        elif len(temp) == 0:
            return None
        else:
            return clean_salary(tuple(temp))
    else:
        return None

In [24]:
def extract_salary_info_from_result(result):
    values = result.findAll('span',class_='no-wrap')
    if values != None:
        # There are a number of salaries that are ranges, so I am spliting on white space and keeping
        # just the values that have a dollar sign.
        temp = list(map(lambda a: list(filter(lambda b: ('year' in b) or ('hour' in b) or ('day' in b) or ('week' in b) or ('month' in b), a.get_text().split(' '))),values))
        
        # Getting the first
        if len(temp) != 0:
            temp = temp[0]
            
        if len(temp) == 1:
            return temp[0]
        elif len(temp) == 0:
            return None
        else:
            return tuple(temp)
    else:
        return None

In [7]:

def extract_job_title_from_result(result):
    values = result.find('a', attrs={"data-tn-element":'jobTitle'})
    temp = []
    if values != None:
        temp.append(values["title"])
#         for v in values:
#             if v != None:
#                 #print(type(v))
#                 temp.append(v)
#             else:
#                 temp.append(None)
        if len(temp) == 1:
            return temp[0]
        else:
            return tuple(temp)
    else:
        return None

#xtract_job_title_from_result(result)

In [8]:

def extract_company_from_result(result):
    values = result.find_all(class_='company')
    temp = []
    if values != None:
        for v in values:
            if v != None:
                # I am getting rid of new line characters and removing excess whitespace that seems to pop up.
                temp.append(v.find(itemprop="name").get_text().replace("\n","").strip())
            else:
                temp.append(None)
    else:
        temp.append(None)
    if len(temp) == 1:
        return temp[0]
    else:
        return tuple(temp)

#extract_company_from_result(result)

In [9]:

def extract_description_from_result(result):
    values = result.find(class_='summary')
    temp = []
    if values != None:
        return values.get_text().replace('\n','')
    else:
        return None

In [10]:
def is_sponsored_result(result):
    values=result.find(class_='sponsoredGray')
    if values != None:
        return True
    else:
        return False


In [53]:
from re import sub
from decimal import Decimal

def clean_salary(salary_string):
    # Salaries could come in as a pair or as a single. Singles should be straight strings and pairs should be tuples
    if type(salary_string) == tuple:
        # Using recursion here to parse the individual values if i get more than one.
        parsed = [clean_salary(s) for s in salary_string]
        return np.mean(parsed)
    
    if type(salary_string) in [int, float]:
        return salary_string
    elif salary_string != None:
        value = float(sub(r'[^\d.]', '', salary_string))
        return value
    else:
        return None

In [55]:
import pandas as pd
import time
# I would like to get the cities from my cost of living data set.
df_cost_living = pd.read_csv("~/Workspace/Data/cost_of_living.csv")
df_cost_living['search_location'] = df_cost_living.city

In [56]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 1500 # Set this to a high-value (5000) to generate more results. 
# # Crawling more results, will also take much longer. First test your code on a small number of results and then expand.
# cities =  set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
#     'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
#     'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Baltimore'])

cities = df_cost_living.city[df_cost_living.country == "United States"]

city_dict = {}

df = pd.DataFrame(columns = ['job_title', 'salary', 'location', 'company','description','search_location','salary_info'])
i = 0
j = 0

time1 = time.time()
file_path = "~/Workspace/Data/indeed_scraped.csv"

In [57]:
for city in cities:
    city_dict[city] =[]
    
    print("on: " + city + " time: " + str(time.time() - time1))
    
    city = city.replace(' ','+')
    for start in range(0, max_results_per_city, 10):

        URL = url_template.format(city,start)
        html = requests.get(URL)
        soup = BeautifulSoup(html.text)
        #city_dict[city].append(soup.find_all(class_=' row result'))
        
        # iterating through row results
        for r in soup.find_all(class_='result'):
            
            if not is_sponsored_result(r):
                df.loc[i] = [extract_job_title_from_result(r),
                            extract_salary_from_result(r),
                            extract_location_from_result(r),
                            extract_company_from_result(r),
                            extract_description_from_result(r),
                            city.replace('+',' '),
                            extract_salary_info_from_result(r)] #replacing + with whitespcae
                # index rows with i count
                i+=1
                # short sleep to stop over hitting the server
                time.sleep(.001)
                
                #saving csv occassionally. jumping by 500s.
                if i % 500 == 0:
                    if j == 0:
                        # if first selection I am writing overtop existing file.
                         df.iloc[j:j+i].to_csv(file_path, index=False)
                    else:
                        # appending if it is not the first write of this selection
                        df.iloc[j:j+i].to_csv(file_path,mode='a', header=False, index=False)
                    # updating j
                    j+=1

time2 = time.time()
print(time2 - time1)

on: New York City time: 24.983976125717163


/home/roland/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/roland/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


on: San Francisco time: 85.34972476959229
on: Washington D.C. time: 126.25852942466736
on: Honolulu time: 164.37589836120605
on: San Jose time: 196.8243613243103
on: Boston time: 245.35741329193115
on: Maui time: 285.38808941841125
on: Oakland time: 315.600869178772
on: Los Angeles time: 356.95254158973694
on: Seattle time: 398.1918468475342
on: San Diego time: 453.5450847148895
on: Chicago time: 498.24929690361023
on: Boulder time: 540.5994446277618
on: Philadelphia time: 581.3341801166534
on: Sacramento time: 623.7558052539825
on: Portland time: 672.1765425205231
on: Anchorage time: 715.8028628826141
on: Denver time: 770.6614933013916
on: Miami time: 830.8518986701965
on: Minneapolis - St. Paul time: 873.7945756912231
on: Baltimore time: 939.1504876613617
on: Providence time: 996.8945245742798
on: Atlanta time: 1052.824295282364
on: Hartford time: 1117.3798699378967
on: Burlington time: 1169.0015277862549
on: Tacoma time: 1222.9123373031616
on: Fort Lauderdale time: 1276.775620222091

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [50]:
# df = pd.DataFrame(columns = ['job_title', 'salary', 'location', 'company','description','search_location','salary_info'])
# i = 0
# for k in city_dict.keys():
#     for row in city_dict[k]:
#         for r in row:

#             if not is_sponsored_result(r):
#                 df.loc[i] = [extract_job_title_from_result(r),
#                             extract_salary_from_result(r),
#                             extract_location_from_result(r),
#                             extract_company_from_result(r),
#                             extract_description_from_result(r),
#                             k.replace('+',' '),
#                             extract_salary_info_from_result(r)] #replacing + with whitespcae
#                 # index rows with i count
#                 i+=1

In [51]:
df.head()

,job_title,salary,location,company,description,search_location,salary_info
0,Machine Learning Engineer (Associate) - Intell...,NaN,"New York, NY 10001 (Chelsea area)",JP Morgan Chase,JP Morgan Intelligent Solutions (JPMIS) is a n...,New York City,None
1,Data Scientist,NaN,"New York, NY",GameChanger,"As a Data Analyst, you’ll transform the teraby...",New York City,None
2,Data Scientist,75000.0,"New York, NY 10022 (Midtown area)",First Derivatives,Data Scientists explore vast amounts of inform...,New York City,year
3,"Data Scientist, Forecast Analyst",NaN,"New York, NY",NBCUniversal,Experience with data visualization techniques ...,New York City,None
4,"Data Scientist, Analytics",NaN,"New York, NY",Facebook,The Data Scientist Analytics role has work acr...,New York City,None


Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries

In [29]:
df.drop_duplicates(inplace=True)
df[ [d for d in df.salary_info.isnull()]]
# I will deal with weekly and hourly salaries later.

,job_title,salary,location,company,description,search_location,salary_info
0,Machine Learning Engineer (Associate) - Intell...,None,"New York, NY 10001 (Chelsea area)",JP Morgan Chase,JP Morgan Intelligent Solutions (JPMIS) is a n...,New York City,None
1,Data Scientist,None,"New York, NY",GameChanger,"As a Data Analyst, you’ll transform the teraby...",New York City,None
3,"Data Scientist, Forecast Analyst",None,"New York, NY",NBCUniversal,Experience with data visualization techniques ...,New York City,None
4,"Data Scientist, Analytics",None,"New York, NY",Facebook,The Data Scientist Analytics role has work acr...,New York City,None
5,Data Scientist - Test & Learn Analytics,None,"New York, NY 10003 (Greenwich Village area)","J.Crew Group, Inc.",Data Scientist – Test and Learn Analytics. Han...,New York City,None
6,Data Scientist,None,"Manhattan, NY",Diaspark,"Team-oriented (we run a meritocracy), creative...",New York City,None
8,Data Scientist,None,"New York, NY",Blue State Digital,Use data to build a better world. You will be ...,New York City,None
9,Data Scientist,None,"New York, NY 10012 (Little Italy area)",Meetup,"3+ years working with data analytics, data war...",New York City,None
10,Data Scientist,None,"New York, NY",ThreatMetrix,"As a Data Scientist, you’ll use data from the ...",New York City,None
11,Data Scientist - Pricing,None,"New York, NY 10003 (Greenwich Village area)","J.Crew Group, Inc.",J.Crew is seeking a Data Scientist with an emp...,New York City,None


#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [31]:
# Converting salary column to numbers, averaging over a range.
df['salary'] = df.salary.apply(clean_salary)

In [33]:
df_clean = df[df.salary.isnull() == False]
df_clean.sort_values('salary')
# Merging my cost of living data with my dataframe.
# Sub selecting cost of living data for easier formatting.
df_cost_living = df_cost_living[['search_location','cost_index']]
df_clean = df_clean.merge(df_cost_living, on='search_location', how='left')

In [34]:
df_clean['salary_adjusted'] = salary_adjust(df_clean.salary, df_clean.salary_info)
df_clean_simple = df_clean[df_clean.salary > 20000]

In [35]:
df_clean.describe()

,salary,cost_index,salary_adjusted
count,824.000000,824.000000,824.000000
mean,73694.608076,187.519417,91644.296602
std,58076.281422,36.539226,45833.854554
min,10.010000,125.000000,20820.800000
25%,6499.750000,161.000000,55817.875000
50%,70399.500000,183.000000,80000.000000
75%,113125.000000,213.000000,120000.000000
max,275000.000000,251.000000,275000.000000


In [36]:
df_clean_simple.describe()

,salary,cost_index,salary_adjusted
count,607.000000,607.000000,607.000000
mean,99517.644152,190.581549,99517.644152
std,45190.206843,38.093894,45190.206843
min,22500.000000,125.000000,22500.000000
25%,64500.000000,161.000000,64500.000000
50%,91166.500000,186.000000,91166.500000
75%,129412.500000,234.000000,129412.500000
max,275000.000000,251.000000,275000.000000


When adjusting salaries I get more data points, but the average salary becomes lower. I think this is still a valid approach to make. And I still have the data marked to determine which values are adjusted and which are not.

### Save your results as a CSV

In [49]:
df_clean.groupby('search_location').count().sort_values('salary', ascending=False)['job_title'].head(10)

search_location
New York City      88
Washington D.C.    59
Tacoma             36
Boston             35
San Francisco      34
San Jose           32
Chicago            31
Oakland            30
Seattle            29
Atlanta            26
Name: job_title, dtype: int64

## Predicting salaries using Random Forests + Another Classifier

#### Load in the the data of scraped salaries

In [ ]:
## YOUR CODE HERE

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't _have_ to choose the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries, 

In [ ]:
## YOUR CODE HERE

#### Thought experiment: What is the baseline accuracy for this model?

In [ ]:
## YOUR CODE HERE

#### Create a Random Forest model to predict High/Low salary using Sklearn. Start by ONLY using the location as a feature. 

In [ ]:
## YOUR CODE HERE

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title or whether 'Manager' is in the title. 
- Then build a new Random Forest with these features. Do they add any value?
- After creating these variables, use count-vectorizer to create features based on the words in the job titles.
- Build a new random forest model with location and these new features included.

In [ ]:
## YOUR CODE HERE

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy of the model, as well as any other metrics you feel are appropriate. 

In [ ]:
## YOUR CODE HERE

#### Repeat the model-building process with a non-tree-based method.

In [ ]:
## YOUR CODE HERE

### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the job descriptions. 
- Examine using count or binary features in the model
- Re-evaluate your models using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [ ]:
## YOUR CODE HERE